In [1]:
"""
DEEP EDA: RM GROUPS & TRAINING TIMEFRAME ANALYSIS
Goal: Understand what we're doing with different RM groups and find optimal training period
"""

import numpy as np
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

receivals = pd.read_csv('./Project_materials/data/kernel/receivals.csv')
prediction_mapping = pd.read_csv('./Project_materials/data/prediction_mapping.csv')

receivals['date_arrival'] = pd.to_datetime(receivals['date_arrival'], utc=True).dt.tz_localize(None)
prediction_mapping['forecast_start_date'] = pd.to_datetime(prediction_mapping['forecast_start_date'])
prediction_mapping['forecast_end_date'] = pd.to_datetime(prediction_mapping['forecast_end_date'])

receivals = receivals[receivals['net_weight'] > 0]
receivals = receivals[receivals['rm_id'].notna()]

print("="*80)
print("PART 1: RM GROUP ANALYSIS")
print("="*80)

# ============================================================================
# 1. CATEGORIZE RMs BY ACTIVITY PATTERN
# ============================================================================
print("\n[1] RM CATEGORIZATION BY ACTIVITY")
print("-"*80)

forecast_start = pd.to_datetime('2025-01-01')
test_rms = prediction_mapping['rm_id'].unique()

rm_profiles = []

for rm_id in test_rms:
    hist = receivals[receivals['rm_id'] == rm_id]
    
    if len(hist) == 0:
        rm_profiles.append({
            'rm_id': rm_id,
            'category': 'no_history',
            'total_deliveries': 0,
            'total_weight': 0,
            'last_delivery': None,
            'days_since_last': 99999,
            'deliveries_2024': 0,
            'weight_2024': 0,
            'deliveries_2023': 0,
            'weight_2023': 0,
            'avg_delivery_size': 0
        })
        continue
    
    last_delivery = hist['date_arrival'].max()
    days_since = (forecast_start - last_delivery).days
    
    hist_2024 = hist[hist['date_arrival'].dt.year == 2024]
    hist_2023 = hist[hist['date_arrival'].dt.year == 2023]
    
    # Categorize
    if len(hist_2024) > 0:
        category = 'active_2024'
    elif len(hist_2023) > 0:
        category = 'active_2023'
    elif days_since < 730:  # Last 2 years
        category = 'recent_inactive'
    elif days_since < 1825:  # Last 5 years
        category = 'old_inactive'
    else:
        category = 'very_old_inactive'
    
    rm_profiles.append({
        'rm_id': rm_id,
        'category': category,
        'total_deliveries': len(hist),
        'total_weight': hist['net_weight'].sum(),
        'last_delivery': last_delivery,
        'days_since_last': days_since,
        'deliveries_2024': len(hist_2024),
        'weight_2024': hist_2024['net_weight'].sum(),
        'deliveries_2023': len(hist_2023),
        'weight_2023': hist_2023['net_weight'].sum(),
        'avg_delivery_size': hist['net_weight'].mean()
    })

rm_df = pd.DataFrame(rm_profiles)

print("\nRM Categories:")
print(rm_df['category'].value_counts())

print("\nTest predictions by category:")
test_with_category = prediction_mapping.merge(rm_df[['rm_id', 'category']], on='rm_id')
category_counts = test_with_category['category'].value_counts()
print(category_counts)
print(f"\nPercentage breakdown:")
for cat in category_counts.index:
    pct = category_counts[cat] / len(test_with_category) * 100
    print(f"  {cat}: {pct:.1f}%")

# ============================================================================
# 2. WHAT DID EACH CATEGORY DO IN 2024?
# ============================================================================
print("\n[2] 2024 BEHAVIOR BY CATEGORY")
print("-"*80)

for category in rm_df['category'].unique():
    cat_rms = rm_df[rm_df['category'] == category]
    print(f"\n{category.upper()} ({len(cat_rms)} RMs):")
    print(f"  Total deliveries in 2024: {cat_rms['deliveries_2024'].sum()}")
    print(f"  Total weight in 2024: {cat_rms['weight_2024'].sum():,.0f} kg")
    print(f"  Avg deliveries per RM: {cat_rms['deliveries_2024'].mean():.1f}")
    print(f"  Avg weight per RM: {cat_rms['weight_2024'].mean():,.0f} kg")
    
    if cat_rms['deliveries_2024'].sum() > 0:
        avg_per_delivery = cat_rms['weight_2024'].sum() / cat_rms['deliveries_2024'].sum()
        print(f"  Avg per delivery: {avg_per_delivery:,.0f} kg")

# ============================================================================
# 3. STEP 5 BEHAVIOR BY CATEGORY
# ============================================================================
print("\n[3] STEP 5 MODEL BEHAVIOR BY CATEGORY")
print("-"*80)

# Simulate Step 5 predictions for each category
print("\nFor each category, Step 5 would predict:")

for category in ['active_2024', 'active_2023', 'recent_inactive', 'old_inactive', 'very_old_inactive']:
    cat_rms = rm_df[rm_df['category'] == category]
    if len(cat_rms) == 0:
        continue
    
    print(f"\n{category.upper()}:")
    
    # Check guardrail behavior
    over_365_days = (cat_rms['days_since_last'] > 365).sum()
    between_180_365 = ((cat_rms['days_since_last'] > 180) & (cat_rms['days_since_last'] <= 365)).sum()
    under_180 = (cat_rms['days_since_last'] <= 180).sum()
    
    print(f"  RMs with days_since > 365: {over_365_days} → forced to 0")
    print(f"  RMs with 180 < days_since <= 365: {between_180_365} → cold cap applied")
    print(f"  RMs with days_since <= 180: {under_180} → normal prediction")
    
    # Estimate what model would predict
    predictions_for_category = len(cat_rms) * 150  # Average horizon
    predictions_forced_zero = over_365_days * 150
    
    print(f"  Total predictions for this category: {predictions_for_category}")
    print(f"  Predictions forced to zero: {predictions_forced_zero} ({predictions_forced_zero/predictions_for_category*100:.1f}%)")

# ============================================================================
# 4. WHAT SHOULD WE PREDICT FOR EACH CATEGORY?
# ============================================================================
print("\n[4] EXPECTED BEHAVIOR IN TEST (JAN-MAY 2025)")
print("-"*80)

print("\nHypothesis for each category:")

print("\nACTIVE_2024 RMs:")
print("  - Had deliveries throughout 2024")
print("  - Likely to continue in 2025")
print("  - Should predict based on 2024 rates")
print("  - Problem: We might be UNDER-predicting these if Jan-May > Sep-Nov")

print("\nACTIVE_2023 RMs:")
print("  - Active in 2023 but not 2024")
print("  - Might restart in 2025, might not")
print("  - Current model: forced to 0 (days_since > 365)")
print("  - Question: Should we predict something small?")

print("\nRECENT/OLD/VERY_OLD INACTIVE RMs:")
print("  - Haven't delivered in years")
print("  - Probably won't deliver in 2025")
print("  - Current model: forced to 0")
print("  - This is probably CORRECT")

# ============================================================================
# PART 2: TRAINING TIMEFRAME ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("PART 2: TRAINING TIMEFRAME ANALYSIS")
print("="*80)

# ============================================================================
# 5. AVAILABLE DATA BY YEAR
# ============================================================================
print("\n[5] AVAILABLE TRAINING DATA")
print("-"*80)

yearly_stats = receivals.groupby(receivals['date_arrival'].dt.year).agg({
    'net_weight': ['sum', 'count'],
    'rm_id': 'nunique'
}).round(0)
yearly_stats.columns = ['total_weight', 'deliveries', 'unique_rms']
print("\nData by year:")
print(yearly_stats)

print("\nData by year for TEST RMs only:")
test_rm_data = receivals[receivals['rm_id'].isin(test_rms)]
yearly_test = test_rm_data.groupby(test_rm_data['date_arrival'].dt.year).agg({
    'net_weight': ['sum', 'count'],
    'rm_id': 'nunique'
}).round(0)
yearly_test.columns = ['total_weight', 'deliveries', 'unique_rms']
print(yearly_test)

# ============================================================================
# 6. CURRENT TRAINING SETUP
# ============================================================================
print("\n[6] CURRENT TRAINING SETUP (2024 only)")
print("-"*80)

train_2024 = receivals[receivals['date_arrival'].dt.year == 2024]
print(f"Training data: Jan-Nov 2024")
print(f"  Deliveries: {len(train_2024)}")
print(f"  Total weight: {train_2024['net_weight'].sum():,.0f} kg")
print(f"  Unique RMs: {train_2024['rm_id'].nunique()}")
print(f"  Average per delivery: {train_2024['net_weight'].mean():,.0f} kg")

# ============================================================================
# 7. ALTERNATIVE TRAINING WINDOWS
# ============================================================================
print("\n[7] ALTERNATIVE TRAINING WINDOWS")
print("-"*80)

# Option 1: Last 2 years
cutoff_2years = forecast_start - timedelta(days=730)
train_2years = receivals[receivals['date_arrival'] >= cutoff_2years]
print(f"\nOption 1: Last 2 years (since {cutoff_2years.date()})")
print(f"  Deliveries: {len(train_2years)}")
print(f"  Total weight: {train_2years['net_weight'].sum():,.0f} kg")
print(f"  Unique RMs: {train_2years['rm_id'].nunique()}")
print(f"  RMs in test that appear: {len(set(train_2years['rm_id']) & set(test_rms))}")

# Option 2: Last 3 years
cutoff_3years = forecast_start - timedelta(days=1095)
train_3years = receivals[receivals['date_arrival'] >= cutoff_3years]
print(f"\nOption 2: Last 3 years (since {cutoff_3years.date()})")
print(f"  Deliveries: {len(train_3years)}")
print(f"  Total weight: {train_3years['net_weight'].sum():,.0f} kg")
print(f"  Unique RMs: {train_3years['rm_id'].nunique()}")
print(f"  RMs in test that appear: {len(set(train_3years['rm_id']) & set(test_rms))}")

# Option 3: All historical data
print(f"\nOption 3: All historical data")
print(f"  Deliveries: {len(receivals)}")
print(f"  Total weight: {receivals['net_weight'].sum():,.0f} kg")
print(f"  Unique RMs: {receivals['rm_id'].nunique()}")
print(f"  RMs in test that appear: {len(set(receivals['rm_id']) & set(test_rms))}")

# ============================================================================
# 8. COVERAGE ANALYSIS
# ============================================================================
print("\n[8] TRAINING DATA COVERAGE OF TEST RMs")
print("-"*80)

coverage_by_window = []

for window_name, cutoff in [
    ("2024 only", pd.to_datetime('2024-01-01')),
    ("Last 2 years", cutoff_2years),
    ("Last 3 years", cutoff_3years),
    ("All time", receivals['date_arrival'].min())
]:
    window_data = receivals[receivals['date_arrival'] >= cutoff]
    window_rms = window_data['rm_id'].unique()
    
    test_coverage = len(set(window_rms) & set(test_rms))
    test_missing = len(set(test_rms) - set(window_rms))
    
    coverage_by_window.append({
        'window': window_name,
        'test_rms_covered': test_coverage,
        'test_rms_missing': test_missing,
        'coverage_pct': test_coverage / len(test_rms) * 100
    })

coverage_df = pd.DataFrame(coverage_by_window)
print("\nTest RM coverage by training window:")
print(coverage_df.to_string(index=False))

# ============================================================================
# 9. SEASONALITY: JAN-MAY PATTERNS
# ============================================================================
print("\n[9] SEASONALITY: JAN-MAY HISTORICAL PATTERNS")
print("-"*80)

# For active_2024 RMs, compare different periods
active_2024_rms = rm_df[rm_df['category'] == 'active_2024']['rm_id'].values

if len(active_2024_rms) > 0:
    active_data = receivals[receivals['rm_id'].isin(active_2024_rms)]
    
    # Jan-May patterns by year
    jan_may_data = active_data[active_data['date_arrival'].dt.month.isin([1,2,3,4,5])]
    
    print("\nJan-May deliveries by year (for active_2024 RMs):")
    jan_may_yearly = jan_may_data.groupby(jan_may_data['date_arrival'].dt.year).agg({
        'net_weight': 'sum'
    }).round(0)
    print(jan_may_yearly)
    
    # Sep-Nov patterns by year
    sep_nov_data = active_data[active_data['date_arrival'].dt.month.isin([9,10,11])]
    
    print("\nSep-Nov deliveries by year (for active_2024 RMs):")
    sep_nov_yearly = sep_nov_data.groupby(sep_nov_data['date_arrival'].dt.year).agg({
        'net_weight': 'sum'
    }).round(0)
    print(sep_nov_yearly)
    
    # Compare if we have both periods
    if len(jan_may_yearly) > 0 and len(sep_nov_yearly) > 0:
        common_years = set(jan_may_yearly.index) & set(sep_nov_yearly.index)
        if len(common_years) > 0:
            print("\nJan-May vs Sep-Nov ratio (for years with both):")
            for year in sorted(common_years):
                jan_may = jan_may_yearly.loc[year, 'net_weight']
                sep_nov = sep_nov_yearly.loc[year, 'net_weight']
                ratio = jan_may / sep_nov if sep_nov > 0 else 0
                print(f"  {year}: Jan-May is {ratio:.2f}x Sep-Nov")

# ============================================================================
# 10. KEY INSIGHTS
# ============================================================================
print("\n" + "="*80)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("="*80)

# Active 2024 analysis
active_2024_count = len(rm_df[rm_df['category'] == 'active_2024'])
active_2024_predictions = len(test_with_category[test_with_category['category'] == 'active_2024'])

# Inactive analysis
inactive_categories = ['active_2023', 'recent_inactive', 'old_inactive', 'very_old_inactive']
inactive_count = len(rm_df[rm_df['category'].isin(inactive_categories)])
inactive_predictions = len(test_with_category[test_with_category['category'].isin(inactive_categories)])

print(f"\n1. RM GROUP BREAKDOWN:")
print(f"   - Active 2024: {active_2024_count} RMs, {active_2024_predictions} predictions ({active_2024_predictions/len(test_with_category)*100:.1f}%)")
print(f"   - Inactive: {inactive_count} RMs, {inactive_predictions} predictions ({inactive_predictions/len(test_with_category)*100:.1f}%)")

print(f"\n2. TRAINING DATA:")
print(f"   - Current (2024 only): Covers {coverage_df.iloc[0]['test_rms_covered']}/{len(test_rms)} test RMs ({coverage_df.iloc[0]['coverage_pct']:.1f}%)")
print(f"   - All time: Covers {coverage_df.iloc[3]['test_rms_covered']}/{len(test_rms)} test RMs ({coverage_df.iloc[3]['coverage_pct']:.1f}%)")

print(f"\n3. THE PROBLEM:")
print(f"   - You're training on 2024 data (Jan-Nov)")
print(f"   - Validation is Sep-Nov 2024")
print(f"   - Test is Jan-May 2025")
print(f"   - These periods might have DIFFERENT throughput!")

print(f"\n4. HYPOTHESES TO TEST:")
print(f"   A. Use longer training window (2-3 years) to capture more patterns")
print(f"   B. Create separate models for active_2024 vs inactive RMs")
print(f"   C. Validate on Jan-May 2024 instead of Sep-Nov 2024")
print(f"   D. Accept that inactive RMs should be 0 (Step 5 is correct)")

print("\n" + "="*80)

PART 1: RM GROUP ANALYSIS

[1] RM CATEGORIZATION BY ACTIVITY
--------------------------------------------------------------------------------

RM Categories:
category
very_old_inactive    110
active_2024           60
old_inactive          21
active_2023           12
Name: count, dtype: int64

Test predictions by category:
category
very_old_inactive    16500
active_2024           9000
old_inactive          3150
active_2023           1800
Name: count, dtype: int64

Percentage breakdown:
  very_old_inactive: 54.2%
  active_2024: 29.6%
  old_inactive: 10.3%
  active_2023: 5.9%

[2] 2024 BEHAVIOR BY CATEGORY
--------------------------------------------------------------------------------

VERY_OLD_INACTIVE (110 RMs):
  Total deliveries in 2024: 0
  Total weight in 2024: 0 kg
  Avg deliveries per RM: 0.0
  Avg weight per RM: 0 kg

ACTIVE_2024 (60 RMs):
  Total deliveries in 2024: 5996
  Total weight in 2024: 86,101,113 kg
  Avg deliveries per RM: 99.9
  Avg weight per RM: 1,435,019 kg
  Avg 